In [1]:
!pip install hypothesis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import string 
import random 
from numpy.random import choice
from scipy.stats import skewnorm

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random


import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import next_chars_stuff

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
next_chars_stuff.regex_to_nfa('\\?')


{<next_chars_stuff.Node at 0x7fa81e795bd0>}

In [41]:
import re
if re.search('[0-9]', '9'):
  print('hi')

hi


In [42]:
def generate_training_data(num_examples: int, max_size:int, regex_things:list) -> list:
  examples = [ [ regex_things[random.randint(0,len(regex_things)-1)] for  i in range(random.randint(1,max_size))] for k in range(num_examples)]
  return examples
  
def generate_unambig_training_data(num_examples: int, max_size:int, regex_things:list) -> list:
  examples = [ ]
  for  i in range(num_examples):
    count = 0
    digit_plus = False 
    low_letter_plus = False
    high_letter_plus = False
    alphanum_plus = False
    max_regex_size = random.randint(1,max_size)
    regex = []
    while count < max_regex_size:
      atom = regex_things[random.randint(0,len(regex_things)-1)]
      if digit_plus:
        if re.search(atom, '9') and  re.search(atom, '1'):
          continue
      if low_letter_plus:
        if re.search(atom, 'a') and re.search(atom, 'b'):
          continue
      if high_letter_plus:
        if re.search(atom, 'A') and re.search(atom, 'B') :
          continue
      if high_letter_plus:
        atom = regex_things[10:][random.randint(0,len(regex_things[10:])-1)]
      regex.append(atom)
      digit_plus, low_letter_plus, high_letter_plus,  alphanum_plus  = False , False , False, False
      if atom[-1] == '+':
        if atom == '[0-9]+':
          digit_plus = True 
        elif atom == '[a-z]+':
          low_letter_plus = True
        elif atom == '[A-Z]+':
          high_letter_plus = True
        else:
          alphanum_plus = True
      count+=1
      continue 
    examples.append(regex)
  return examples
 
def generate_training_data_pairs(training_data: list) -> list:
  #for i in training_data:
    #print(i)
    #hypothesis.strategies.from_regex('^' + ''.join(i) + '$').example().strip()
  return [(i,i) for i in training_data]
  #return [(i, hypothesis.strategies.from_regex('^' + ''.join(i) + '$').example().strip()) for i in training_data]

In [43]:
# make the language 
# first iteration = very restricted language no optionals no constants
regex_things : list = ['[0-9]','[a-z]','[A-Z]','[a-zA-Z]', '[a-zA-Z0-9]', '[0-9]+','[a-z]+','[A-Z]+','[a-zA-Z]+', '[a-zA-Z0-9]+']
ascii_char : list = list(string.printable)[:95]
special_char = ['!','"','#','%','&',"'",',','-','.',':',';',
 '<','>','@','_','`',' ']
#regex_things = regex_things + [ i for i in ascii_char if i not in special_char]
regex_things = regex_things + ascii_char[:65] + special_char
# takes some time to run 
# TODO: Write this into a file and read from a file
training_data = generate_unambig_training_data(10000, 10, regex_things)
pairs = generate_training_data_pairs(training_data)

In [44]:
SOS_token = 0
EOS_token = 1

# constructing langiage using 
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
      
    def addList(self, list_words):
        for word in list_words:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [45]:
MAX_LENGTH = 12
def readLangs(lang1: list, lang2:list, reverse=False):
  input_lang = Lang('regex')
  output_lang = Lang('english')
  input_lang.addList(lang1)
  output_lang.addList(lang2)
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  return input_lang, output_lang


In [46]:
input_lang, output_lang = readLangs(regex_things, ascii_char)

regex 91
english 97


In [47]:
# model code also taken from 
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [48]:
def indexesFromList(lang, char_list):
    return [lang.word2index[word] for word in char_list]


def tensorFromList(lang, char_list):
    indexes = indexesFromList(lang, char_list)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def pairFromPair(pair):
    input_tensor = tensorFromList(input_lang, pair[0])
    #target_tensor = tensorFromList(output_lang, list(pair[1]))
    return (pair[0], input_tensor)


In [49]:
teacher_forcing_ratio = 0.5

#changed this to have an input_regex
def train(input_regex, input_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    #target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    #use_teacher_forcing = True 
    #if random.random() < teacher_forcing_ratio else False

    # Teacher forcing: Feed the target as the next input
    state = next_chars_stuff.regex_to_nfa(input_regex)
    chosen_token = np.inf
    count = 0
    while chosen_token!=EOS_token and count < 12:
        #print(count)
        count +=1
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        next_chars = next_chars_stuff.possible_next_chars(state)

        indexed_chars = indexesFromList(output_lang, next_chars)
        true_tensor = torch.zeros(output_lang.n_words)
        true_tensor = true_tensor.to(device)
        if next_chars_stuff.end_token_is_allowed_here(state):
          indexed_chars.append(EOS_token)
        for i in indexed_chars:
          true_tensor[i] = 1/ len(indexed_chars) 
        # create a tensor that gives equal probabilities to next chars 
        loss += criterion(decoder_output, true_tensor.reshape(1, output_lang.n_words))
        chosen_token = random.choice(indexed_chars)
        next_tensor = torch.zeros(output_lang.n_words)
        next_tensor[chosen_token] = 1
        if (chosen_token == EOS_token):
            break
        state = next_chars_stuff.consume_a_char(state, output_lang.index2word[chosen_token])
        #print(output_lang.index2word[chosen_token])
        #re.complie(#amount of regex we have consumed so far)
        # 0 or 1
        #chosen_tensor = torch.tensor(chosen_token)
        decoder_input = torch.tensor(chosen_token, device = device)  # Teacher forcing


    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / count

In [50]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [51]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [pairFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.CrossEntropyLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_regex = training_pair[0]
        input_tensor = training_pair[1]

        loss = train(input_regex, input_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [52]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [53]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromList(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            # TODO: Change to output 
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [58]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 35000, print_every=1000)

0m 21s (- 11m 57s) (1000 2%) 4.1427
0m 37s (- 10m 23s) (2000 5%) 3.9165
0m 54s (- 9m 44s) (3000 8%) 3.4492
1m 11s (- 9m 13s) (4000 11%) 2.7078
1m 28s (- 8m 51s) (5000 14%) 2.2338
1m 46s (- 8m 33s) (6000 17%) 2.0150
2m 2s (- 8m 10s) (7000 20%) 1.6524
2m 19s (- 7m 50s) (8000 22%) 1.5779
2m 35s (- 7m 30s) (9000 25%) 1.4157
2m 52s (- 7m 11s) (10000 28%) 1.4913
3m 9s (- 6m 52s) (11000 31%) 1.5052
3m 26s (- 6m 35s) (12000 34%) 1.3286
3m 42s (- 6m 17s) (13000 37%) 1.3178
3m 59s (- 5m 59s) (14000 40%) 1.2968
4m 16s (- 5m 41s) (15000 42%) 1.2148
4m 33s (- 5m 24s) (16000 45%) 1.4462
4m 49s (- 5m 6s) (17000 48%) 1.2814
5m 6s (- 4m 49s) (18000 51%) 1.2829
5m 23s (- 4m 32s) (19000 54%) 1.1283
5m 39s (- 4m 14s) (20000 57%) 1.1199
5m 56s (- 3m 57s) (21000 60%) 1.1541
6m 13s (- 3m 40s) (22000 62%) 1.0926
6m 30s (- 3m 23s) (23000 65%) 1.0860
6m 47s (- 3m 6s) (24000 68%) 0.9696
7m 4s (- 2m 49s) (25000 71%) 1.1011
7m 20s (- 2m 32s) (26000 74%) 0.9485
7m 37s (- 2m 15s) (27000 77%) 1.0615
7m 53s (- 1m 58s)

In [60]:
evaluate(encoder1, attn_decoder1, ['[0-9]', '[a-z]+'])

(['4', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r'],
 tensor([[5.0378e-09, 9.9996e-01, 9.6521e-08, 5.4667e-14, 5.1724e-12, 1.0692e-12,
          3.9238e-05, 1.3690e-08, 1.2106e-13, 2.0089e-08, 1.7122e-08, 5.4568e-09],
         [3.3047e-09, 1.7612e-08, 1.0000e+00, 2.1735e-07, 4.5435e-16, 3.6468e-14,
          6.7024e-14, 5.2267e-10, 1.8099e-06, 4.5044e-08, 1.0034e-08, 1.4299e-09],
         [7.5926e-08, 9.9566e-07, 9.9979e-01, 2.9811e-05, 3.0100e-10, 5.3053e-10,
          3.5365e-11, 4.7665e-13, 1.7581e-04, 9.7736e-07, 2.8842e-07, 7.0209e-08],
         [6.1795e-04, 5.7682e-03, 3.1133e-01, 2.3623e-03, 6.3977e-04, 6.7170e-05,
          1.4678e-04, 4.1601e-10, 6.2645e-01, 4.8408e-02, 3.4220e-03, 7.8747e-04],
         [7.6596e-04, 6.0733e-03, 1.0696e-01, 1.0877e-03, 4.8574e-04, 3.0093e-04,
          2.4939e-04, 3.0655e-09, 8.5287e-01, 2.5766e-02, 4.3884e-03, 1.0483e-03],
         [1.5036e-03, 1.5299e-02, 2.7195e-01, 4.0184e-03, 2.6183e-03, 4.7342e-04,
          3.7118e-04, 3.9339e-

In [78]:
testing_data = generate_unambig_training_data(1000, 10, regex_things)
count = 0
for regex_list in testing_data:
  answer = evaluate(encoder1, attn_decoder1, regex_list)
  print(regex_list)
  print(answer[0])
  if re.search('^' + ''.join(regex_list) + '$', ''.join(answer[0][:-1])):
      count+=1
      print("CORRECT")
print((count/1000) * 100)


['[a-zA-Z0-9]', '<', 'K', '!', '[a-z]+', '[0-9]+', 'Z', 'b', 'd']
['<', 'K', '<', '!', 'i', 'r', 'r', 'r', 'r', 'r', 'r', 'r']
['>']
['>', '<EOS>']
CORRECT
['p', '3']
['p', '3', '<EOS>']
CORRECT
['A']
['A', '<EOS>']
CORRECT
['>', '[a-zA-Z0-9]+']
['>', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r']
CORRECT
['O', '_', '-']
['O', '_', '-', '<EOS>']
CORRECT
['5', 'r', 'u', '[a-zA-Z0-9]', '[0-9]', 't']
['5', '5', 'u', 'r', '5', 't', '<EOS>']
["'", '#', '"', 'P', 'h', 'w', 'T', '7', 'L']
["'", '#', '"', 'P', 'h', 'w', 'T', '7', 'L', '<EOS>']
CORRECT
['b', "'", '8', '[a-zA-Z]', '.', '`', 'c', '[a-zA-Z0-9]']
['b', "'", '8', 'H', '.', '`', 'c', 'e', '<EOS>']
CORRECT
['#', '6', '1', '[a-zA-Z]+', '!', '%', '>']
['#', '6', '1', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'r']
['e', '2', 'c', '@', '"', 'w']
['e', '2', 'c', '@', '"', 'w', '<EOS>']
CORRECT
['N', '<', 'Y', 'R', '[A-Z]+', '0']
['N', '<', 'Y', 'R', 'Z', 'M', 'C', 'K', 'D', 'D', 'Z', 'L']
['p', 'c', 'H', '"', "'", 'q', 'R', '&']
['p'

KeyboardInterrupt: ignored

In [ ]:

'$', ''.join(answer)